In [1]:
# install the dbldatagen library
%pip install dbldatagen


StatementMeta(, , , Waiting, )

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.3/86.3 kB 14.4 MB/s eta 0:00:00


Note: you may need to restart the kernel to use updated packages.


In [3]:
# install the Faker library
%pip install Faker

StatementMeta(, , , Waiting, )

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 78.4 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.0 -> 23.2.1
[notice] To update, run: /nfs4/pyenv-2daeef60-ddb4-4f40-a89d-7fda4c10b143/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
# Sixth Synthetic data generation exercice

import dbldatagen as dg #import the library
from faker import Faker #import the library
from pyspark.sql.types import IntegerType, FloatType, StringType,DateType,TimestampType  #use sql types to define the type of the columns.
from dbldatagen import PyfuncText#this will help us calling the Faker library from within the DataGenerator object
import dbldatagen.distributions as dist



uniqueCustomers=2000


name_generator = (lambda context, v : Faker(locale="en_US").name()) #create a lambda function to be called from the DataGeneration object. This function will create the Fake Name
email_generator = (lambda context, v : Faker(locale="en_US").ascii_company_email()) #create a lambda function to be called from the DataGeneration object.  This function will create the Fake Email
ip_address_generator = (lambda context, v : Faker(locale="en_US").ipv4_private())#This function will create a fake Ip Address
address_generator = (lambda context, v : Faker(locale="en_US").address())#This function will create a fake address

ds = (dg.DataGenerator(spark, rows=1000, partitions=4) #1000 rows of data
            .withColumn("customer_id",IntegerType(), uniqueValues=uniqueCustomers) #create a custom ID for the customer.  Limited to a specific number of customers
            .withColumn("name",StringType(), percentNulls=0.01, text=PyfuncText(name_generator))
            .withColumn("address",StringType(), percentNulls=0.01, text=PyfuncText(address_generator))                
            .withColumn("payment_instrument_type",StringType(), 
                values=['Paypal', 'Visa', 'Mastercard', 'American Express', 'Discover', 'Branded visa', 'Branded Mastercard'], random=True, distribution="normal")
            .withColumn("email",StringType(), text=PyfuncText(email_generator))
            .withColumn("categories", IntegerType(), minValue=100, maxValue=200, random=True)#
            .withColumn("attempts", IntegerType(), minValue=1, maxValue=10, random=True)
            .withColumn("items", IntegerType(), uniqueValues=10, random=True, distribution=dist.Gamma(1.0, 2.0))
            .withColumn("purchase_date", DateType(), data_range=dg.DateRange("2017-10-01 00:00:00", "2018-10-06 11:55:00", "days=1"), random=True)
            .withColumn("purchase_id", IntegerType(), minValue=1000000, maxValue=2000000, random=True)
            .withColumn("product_code", IntegerType(), uniqueValues=10000, random=True)
            .withColumn("in_stock", StringType(), values=['yes', 'no', 'unknown'])     
            .withColumn("channel", StringType(), values=['web', 'mobile', 'phone'], random=True)
            .withColumn("manufacturer", StringType(), values=['Delta corp', 'Xyzzy Inc.', 'Lakehouse Ltd', 'Acme Corp', 'Embanks Devices'], random=True)
            .withColumn("event_type", StringType(), values=["activation", "deactivation"], random=True, weights=[9, 1])
            .withColumn("ip_address",StringType(), template=r'\\n.\\n.\\n.\\n')
            .withColumn("sample_text",StringType(), text=dg.ILText(paragraphs=(1, 4),sentences=(2, 6)))
            .withColumn("customer_notes",StringType(), text=dg.ILText(words=(1,8)))
            .withColumn("created_ts", TimestampType(), expr="now()")
            .withColumn("modified_ts", TimestampType(), expr="now()")
            .withColumn("memo", StringType(), expr="'original data'")
            )
df = ds.build()
df.write.mode("overwrite").format("delta").saveAsTable("Exercice6")
display(df.limit(100))

StatementMeta(, , , Waiting, )

SynapseWidget(Synapse.DataFrame, 91ab2d5a-1d8e-42f0-affa-872a6aeb1e84)